# Character level language model - Dinosaurus land
### Keras Implementation of the Deep-Learning Specializiation project


Naming new dinos using RNNs

In [1]:
from keras.models import Model
from keras.layers import Input, SimpleRNN, Activation, Dense
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback
from numpy.random import seed
import tensorflow as tf
import random
import sys

/Users/gerardoduran/anaconda/envs/deeplearning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def decode_ix(encoding):
    """
    Decode an encoded dinosaur sequence presented as an
    (n,1) numpy array
    """
    string = "".join(ix_to_char[ix] for ix in encoding.ravel())
    return string

In [3]:
import numpy as np
with open("dinos.txt", "r") as f:
    dinos = f.read().lower()
characters = sorted(list(set(dinos)))

We look at a small sample of the names in the training dataset

In [4]:
seed(1643)
np.random.choice(dinos.split(), 7)

array(['opisthocoelicaudia', 'skeleton', 'marshosaurus', 'tototlmimus',
       'ozraptor', 'hylosaurus', 'elvisaurus'], dtype='<U23')

In [5]:
# Map from all characters in the training dataset to a uniquex index
char_to_ix = {ix:char for char, ix in enumerate(characters)}
# Reverse map of char_to_ix to retrieve the index given the character
ix_to_char = {char:ix for char, ix in enumerate(characters,)}

nchars = len(characters)
nvocab = len(dinos)

In [6]:
char_to_ix

{'\n': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [7]:
# Size of the sequence to pass as training example
seqlen = 10
xtrain, ytrain = [], []
for i in range(0, nvocab - seqlen):
    xt = dinos[i: i + seqlen]
    yt = dinos[i + seqlen]
    xtrain.append([char_to_ix[char] for char in xt])
    ytrain.append([char_to_ix[char] for char in yt])
    
# training dataset is now
# of shape (xtrain X seqlen)
print(f"Number of training instances: {len(xtrain):,}")

# Reshaping into the form:
# nfeatures X timesteps X features
xtrain = np.reshape(xtrain, (-1, seqlen, 1))
# Normalizing values
#xtrain = xtrain / nvocab
# Transorming output values to be
# One-hot encoded
ytrain = to_categorical(ytrain)
xtrain.shape, ytrain.shape

Number of training instances: 19,903


((19903, 10, 1), (19903, 27))

First five instances in the training dataset: A `seqlen` number characters followed by the next character of the dinosaur.

In [8]:
for i in range(5):
    print(xtrain[i].ravel())
    print(repr(decode_ix(xtrain[i])))
    print(repr(ix_to_char[np.where(ytrain[i] == 1)[0][0]]))
    print()

[ 1  1  3  8  5 14 15 19  1 21]
'aachenosau'
'r'

[ 1  3  8  5 14 15 19  1 21 18]
'achenosaur'
'u'

[ 3  8  5 14 15 19  1 21 18 21]
'chenosauru'
's'

[ 8  5 14 15 19  1 21 18 21 19]
'henosaurus'
'\n'

[ 5 14 15 19  1 21 18 21 19  0]
'enosaurus\n'
'a'



In [9]:
tf.reset_default_graph()
### Constructing an RNN model with keras ###
# Input with shape of a single training instance, we
# do not take into account the number of training examples
xin = Input(shape=xtrain.shape[1:])
# A recurrent neural network cell with tanh activation function
# and 256 units
x = SimpleRNN(256, activation="tanh")(xin)
# The output of each training example, after feeding
# sqlen characters is a desne "feedforward" neural network
# with softax activation function
x = Dense(ytrain.shape[1], activation="softmax")(x)

model = Model(inputs=xin, outputs=x)
model.compile(optimizer="adam", loss="categorical_crossentropy")
model.fit(xtrain, ytrain, epochs=100)

Epoch 1/100
19903/19903 [==============================] - 3s 174us/step - loss: 2.4776
Epoch 2/100
19903/19903 [==============================] - 3s 166us/step - loss: 2.3135
Epoch 3/100
19903/19903 [==============================] - 3s 165us/step - loss: 2.2333
Epoch 4/100
19903/19903 [==============================] - 3s 165us/step - loss: 2.1625
Epoch 5/100
19903/19903 [==============================] - 3s 171us/step - loss: 2.1147
Epoch 6/100
19903/19903 [==============================] - 3s 170us/step - loss: 2.0597
Epoch 7/100
19903/19903 [==============================] - 3s 168us/step - loss: 2.0199
Epoch 8/100
19903/19903 [==============================] - 4s 182us/step - loss: 1.9787
Epoch 9/100
19903/19903 [==============================] - 3s 168us/step - loss: 1.9491
Epoch 10/100
19903/19903 [==============================] - 4s 177us/step - loss: 1.9196
Epoch 11/100
19903/19903 [==============================] - 3s 170us/step - loss: 1.8850
Epoch 12/100
19903/19903 [====

19903/19903 [==============================] - 3s 175us/step - loss: 0.9585
Epoch 94/100
19903/19903 [==============================] - 3s 170us/step - loss: 0.9546
Epoch 95/100
19903/19903 [==============================] - 3s 167us/step - loss: 0.9454
Epoch 96/100
19903/19903 [==============================] - 3s 163us/step - loss: 0.9261
Epoch 97/100
19903/19903 [==============================] - 3s 164us/step - loss: 0.9523
Epoch 98/100
19903/19903 [==============================] - 3s 157us/step - loss: 0.9354
Epoch 99/100
19903/19903 [==============================] - 4s 181us/step - loss: 0.9388
Epoch 100/100
19903/19903 [==============================] - 4s 202us/step - loss: 0.9323


In [13]:
# pick a random seed
start = np.random.randint(0, len(xtrain)-1)
pattern = xtrain[start].ravel()
print("Seed:", end="")
print(repr(''.join([ix_to_char[value] for value in pattern])))
# generate characters
for i in range(150):
    x = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = ix_to_char[index]
    seq_in = [ix_to_char[value] for value in pattern]
    print(result, end="")
    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]
print("\n...Done...")

Seed:'us\nhuabeis'
aurus
araucanoraptor
aratososaurus
araucanoraptor
aratososaurus
araucanoraptor
aratososaurus
araucanoraptor
aratososaurus
araucanoraptor
aratososaurus
...Done...


In [ ]:
# Training dataset: Outputs
dinos_list = [[char_to_ix[c] for c in d + "\n"] for d in dinos.split()]
maxlen =  len(max(dinos_list, key=len))
ndinos = len(dinos_list)
nchars = len(characters)
xtrain = np.zeros((ndinos, maxlen, nchars))
ytrain = np.zeros((ndinos, maxlen, nchars))

xtrain = np.zeros((ndinos, maxlen))


In [ ]:
tf.reset_default_graph()
xin = Input(shape=(maxlen,))
x = SimpleRNN(10, activation="tanh")(xin)
x = Dense()
x = Activation("softmax")(x)

model = Model(inputs=xin, outputs=x)
model.compile("adam", loss="categorical_crossentropy")
model.fit(xtrain, ytrain)

In [16]:
model.predict(np.zeros((1, maxlen, nchars)))[0]

array([0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704, 0.03703704, 0.03703704, 0.03703704,
       0.03703704, 0.03703704], dtype=float32)